# Documentation

This notebook demonstrates a workflow for extracting, processing, and querying information from PDF documents using LangChain and Azure OpenAI services.

## Workflow Overview

1. **Data Extraction**  
    - Load PDF documents using `PyPDFLoader`.
    - Each page is treated as a separate `Document`.

2. **Data Processing**  
    - Split documents into manageable text chunks using `RecursiveCharacterTextSplitter`.

3. **Data Ingestion**  
    - Generate embeddings for text chunks using `AzureOpenAIEmbeddings`.
    - Store embeddings in a FAISS vector store for efficient similarity search.

4. **Data Retrieval**  
    - Use a retriever to find relevant document chunks based on user queries.

5. **User Query & RAG (Retrieval-Augmented Generation)**  
    - Format retrieved documents as context.
    - Use a prompt template and `AzureChatOpenAI` to generate answers based on the context.

## Key Variables

- `documents`: List of `Document` objects loaded from the PDF.
- `splitted_doc`: List of text chunks after splitting.
- `azOpenAIembeddings`: Azure OpenAI embeddings model instance.
- `vectorstore`: FAISS vector store containing embedded chunks.
- `retriever`: Retriever object for similarity search.
- `azOpenAIllm`: Azure OpenAI language model instance.
- `prompt`: Prompt template for question answering.
- `rag_chain`: End-to-end chain for RAG-based question answering.

## How to Use

- Update the `file_path` variable to point to your PDF file.
- Run the notebook cells in order to extract, process, and query your document.
- Use the `rag_chain.invoke({"question": "<your question>"})` to get answers from your document.

---
For more details, refer to the comments in each code cell.

## 1. Data Extraction

In [ ]:
import json
from langchain.schema import Document

def documents_to_json(docs: list[Document]):
    return json.dumps([
        {
            "page_content": doc.page_content,
            "metadata": doc.metadata
        } for doc in docs
    ], indent=2)

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader

os.getcwd()

file_path = os.path.join(os.getcwd(), "data", "MLC_user_guide.pdf")

documents = PyPDFLoader(file_path).load() # Each page is a separate document

print(f"Number of pages in the document: {len(documents)}")
print(documents_to_json(documents))

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
)

splitted_doc = text_splitter.split_documents(documents)

print(f"Number of chunks after splitting: {len(splitted_doc)}")
print(documents_to_json(splitted_doc))

## 2. Data Ingestion

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

azOpenAIembeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_version="2023-05-15",
)

# Example usage of the embeddings
query_embeddings = azOpenAIembeddings.embed_query("What is the capital of France?")

In [ ]:
from langchain.vectorstores import FAISS

# FAISS is in memory vector store, so it will not persist across sessions
vectorstore = FAISS.from_documents(
    documents=splitted_doc,
    embedding=azOpenAIembeddings
)

# Example usage of the vector store
relavant_docs = vectorstore.similarity_search("When will your plan start?")

## 3. Data Retrieval

IMPORTANT: We can also convert the vector store into a Retriever object. This makes it easy to integrate with other LangChain methods, as many of them are designed to work with retrievers. Essentially, it serves as a convenient interface.

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs = {"k": 3})

result = retriever.invoke("When will your plan start?")

print(documents_to_json(result))

## 4. User Query

In [ ]:
azOpenAIllm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    api_version="2025-01-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Example usage of the LLM
azOpenAIllm.invoke("When will your plan start?").content

In [ ]:
def format_docs(docs: list[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
    Answer the question based on the context below. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Context: {context}
    Question: {question}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

# LCEL: Langchain Core Expression Language
# This is a runnable chain that takes the context and question, formats the context, and then
# passes it to the prompt, which is then passed to the LLM, and finally parses the output as a string.
# The final output is a string that contains the answer to the question based on the context.

rag_chain = (
    {
        "context": RunnableLambda(lambda x: format_docs(retriever.invoke(x["question"]))),
        "question": RunnablePassthrough()
    }
    | prompt | azOpenAIllm | StrOutputParser()
)

response = rag_chain.invoke({"question": "Could you tell me when will your plan start?"})
print(f"Response: {response}")